In [1]:
import numpy as np
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt
import mpl_finance as mpf
# from mpl_finance import candlestick2_ochl
from tqdm import tqdm


In [2]:
import sys
print(sys.path)

['C:\\Users\\Cyril\\AppData\\Local\\Temp\\3884732e-5348-4960-afbb-4de9d21c535c', 'D:\\Anaconda3\\python37.zip', 'D:\\Anaconda3\\DLLs', 'D:\\Anaconda3\\lib', 'D:\\Anaconda3', '', 'C:\\Users\\Cyril\\AppData\\Roaming\\Python\\Python37\\site-packages', 'D:\\Anaconda3\\lib\\site-packages', 'D:\\Anaconda3\\lib\\site-packages\\win32', 'D:\\Anaconda3\\lib\\site-packages\\win32\\lib', 'D:\\Anaconda3\\lib\\site-packages\\Pythonwin', 'D:\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Cyril\\.ipython']


In [3]:
start_date = '2000-01-01'
end_date = '2015-12-31'

pro = ts.pro_api()

# Obtain  all stocks available

data = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
data = data.dropna()
data.drop(data[data.symbol.astype('int') > 604000].index, inplace = True)
data.head()

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,国农科技,深圳,生物制药,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427


In [4]:
stock_list = [i for i in data.ts_code]
temp = []
for i in stock_list:
    temp.append(i)
    if i.split('.')[0] == '000925':
        break
print(temp[-13])
# Classify industry info
data.industry = data.industry.astype('category')


000910.SZ


In [5]:
all_stocks = tqdm(stock_list)
for stock in all_stocks:
    symbol = stock.split('.')[0]
    df = ts.pro_bar(ts_code=stock, adj='qfq', start_date=start_date, end_date=end_date)
    df.sort_index(axis=0, ascending = False, inplace = True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv('Data/'+symbol+'.csv')


  0%|          | 0/3679 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: 'Data/000001.csv'

In [157]:
start_date = '2000-01-01'
end_date = '2015-12-31'
df = ts.pro_bar(ts_code='000009.SZ', adj='hfq', start_date=start_date , end_date=end_date)
# print(df)

print(df[3050:3110])
# if df is not None:
#     df.sort_index(axis=0, ascending = False, inplace = True)
#     df.reset_index(drop=True, inplace=True)
#     df = df.drop(['pre_close', 'change', 'amount'], axis=1)
#     if len(df)>50:
#         print(df[566:586])

        ts_code trade_date     open     high      low    close  pre_close  \
3050  000009.SZ   20020806  18.8649  20.5488  18.8649  20.0351    18.9220   
3051  000009.SZ   20020805  19.2360  19.2360  18.8649  18.9220    19.2930   
3052  000009.SZ   20020802  19.3216  19.4928  19.1503  19.2930    19.3787   
3053  000009.SZ   20020801  19.0362  19.6070  18.7222  19.3787    19.1218   
3054  000009.SZ   20020731  19.4072  19.6641  19.0362  19.1218    19.4357   
3055  000009.SZ   20020730  19.4643  19.6070  19.0933  19.4357    19.6070   
3056  000009.SZ   20020729  19.2074  19.6355  19.0076  19.6070    19.2930   
3057  000009.SZ   20020726  20.0065  20.0922  19.2645  19.2930    20.0065   
3058  000009.SZ   20020725  20.5488  20.6059  19.8353  20.0065    20.4061   
3059  000009.SZ   20020724  20.0065  20.6059  19.9780  20.4061    19.9495   
3060  000009.SZ   20020723  20.0065  20.3490  19.8068  19.9495    20.2349   
3061  000009.SZ   20020722  20.5773  20.7771  20.1492  20.2349    20.6915   

In [28]:
df1 = df.copy()[:15]
df1.drop([5,6,4,7,8,9,12],inplace = True)
df1.trade_date[10]

'20180815'

In [112]:
def generate_img(data, length):
    data_copy = data[:-length].copy()
    for i in range(len(data)-length):
        start = np.random.choice(data_copy.index)
        img_name = data_copy.trade_date[start]+'_'+data.ts_code[start].split('.')[0]
        data_copy.drop([start], inplace=True)
        img_data = data[start:start+length]
        
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_axes([0,0.2,1,0.8])
        ax2 = fig.add_axes([0,0,1,0.2])
        
#         ax.set_xticks(range(0, len(img_data['trade_date']), 10))
#         ax.set_xticklabels(img_data['trade_date'][::10])
        
        mpf.candlestick2_ochl(ax, img_data['open'], img_data['close'], img_data['high'], img_data['low'],
                            width=0.5, colorup='r', colordown='green',
                            alpha=1)
#         plt.xticks([])
#         plt.yticks([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(False)

        mpf.volume_overlay(ax2, img_data['open'], img_data['close'], img_data['vol'], colorup='r', colordown='g', width=0.5, alpha=0.8)
#         plt.xticks([])
#         plt.yticks([])
        ax2.spines['top'].set_visible(True)
        ax2.spines['right'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        ax2.spines['left'].set_visible(False)
        plt.subplots_adjust(hspace=0)
        
        if data['pct_chg'][start+length] > 2:
            plt.savefig("Data/up/"+img_name + ".png")
        if 0 < data['pct_chg'][start+length] <= 2:
            plt.savefig("Data/up_2/"+img_name + ".png")
        if -2 < data['pct_chg'][start+length] <= 0:
            plt.savefig("Data/down_2/"+img_name + ".png")
        if data['pct_chg'][start+length] <= -2:
            plt.savefig("Data/down/"+img_name + ".png")
        plt.close(fig)

In [113]:
import os

def mkdir(path):

    folder = os.path.exists(path)

    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
        print( "---  new folder...  ---")
        print( "---  OK  ---")

    else:
        print ("---  There is this folder!  ---")

file = "Data/"
mkdir(file+'up')
mkdir(file+'up_2')
mkdir(file+'down')
mkdir(file+'down_2')

generate_img(df,30)

---  There is this folder!  ---
---  There is this folder!  ---
---  There is this folder!  ---
---  There is this folder!  ---
